**Functions to create patterns for phonemes**

Most patterns implemented as example are taken from [A Phonemic-Based Tactile Display for Speech Communication](https://ieeexplore.ieee.org/abstract/document/8423203) by Reed et al. 

In [ ]:
# imports

import json
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, fftshift

With spacing $L/N$, we have the Hann window (also known as $\cos^2$ window): 

$w[n] = \frac{1}{2} (1 - \cos(\frac{2 \pi n}{N})), \quad 0 \leq n \leq N$.

https://en.wikipedia.org/wiki/Hann_function

In [ ]:
def simple_hann_window(coord_list, pho_freq, disc_hann, total_time):
    """
    coord_list: list of coordinates, e.g. [11, 12, 13, 14]
    pho_freq: frequency of phoneme, e.g. 300
    disc_hann: discretization rate of Hanning window, default 24
    total_time: duration of pattern in ms, e.g. 392
    
    returns: list of data needed to create the .json in the right format
    """
    max_amp = 250                   # maximum amplitude, for our motors 250
    window = np.hanning(disc_hann)  # creating normalized hanning window
    cos_window = max_amp*window     # scaling to maximum amplitude
    
    data = []
    for i in range(len(cos_window)):
        iteration = {"iteration": [], "time": 20}
        motor = {"coord": coord_list[0], "amplitude": int(cos_window[i]), "frequency": pho_freq}
        if len(coordlist) == 2:
            motor2 = {"coord": coord_list[1], "amplitude": int(cos_window[i]), "frequency": pho_freq}
        if len(coordlist) == 3:
            motor2 = {"coord": coord_list[1], "amplitude": int(cos_window[i]), "frequency": pho_freq}
            motor3 = {"coord": coord_list[2], "amplitude": int(cos_window[i]), "frequency": pho_freq}
        if len(coordlist) == 4:
            motor2 = {"coord": coord_list[1], "amplitude": int(cos_window[i]), "frequency": pho_freq}
            motor3 = {"coord": coord_list[2], "amplitude": int(cos_window[i]), "frequency": pho_freq}
            motor4 = {"coord": coord_list[3], "amplitude": int(cos_window[i]), "frequency": pho_freq}
        if len(coordlist) == 8:
            motor2 = {"coord": coord_list[1], "amplitude": int(cos_window[i]), "frequency": pho_freq}
            motor3 = {"coord": coord_list[2], "amplitude": int(cos_window[i]), "frequency": pho_freq}
            motor4 = {"coord": coord_list[3], "amplitude": int(cos_window[i]), "frequency": pho_freq}
            motor5 = {"coord": coord_list[4], "amplitude": int(cos_window[i]), "frequency": pho_freq}
            motor6 = {"coord": coord_list[5], "amplitude": int(cos_window[i]), "frequency": pho_freq}
            motor7 = {"coord": coord_list[6], "amplitude": int(cos_window[i]), "frequency": pho_freq}
            motor8 = {"coord": coord_list[7], "amplitude": int(cos_window[i]), "frequency": pho_freq}
        iteration["iteration"].append(motor)
        if len(coordlist) == 2:
            iteration["iteration"].append(motor2)
        if len(coordlist) == 3:
            iteration["iteration"].append(motor2)
            iteration["iteration"].append(motor3)
        if len(coordlist) == 4:
            iteration["iteration"].append(motor2)
            iteration["iteration"].append(motor3)
            iteration["iteration"].append(motor4)
        if len(coordlist) == 8:
            iteration["iteration"].append(motor2)
            iteration["iteration"].append(motor3)
            iteration["iteration"].append(motor4)
            iteration["iteration"].append(motor5)
            iteration["iteration"].append(motor6)
            iteration["iteration"].append(motor7)
            iteration["iteration"].append(motor8)
        data.append(iteration)
        
    return data
    

In [ ]:
def create_simple_hann_pattern(phoneme, coord_list, pho_freq, disc_hann, total_time):
    """
    phoneme: name of phoneme, e.g. 'CH'
    coord_list: list of coordinates, e.g. [11, 12, 13, 14]
    pho_freq: frequency of phoneme, e.g. 300
    disc_hann: discretization rate of Hanning window, default 24
    total_time: duration of pattern in ms, e.g. 392
    
    returns: creates .json file of phoneme
    """
    all_waves = simple_hann_window(coord_list, pho_freq, disc_hann, total_time)
    json_pattern = {"pattern" : all_waves}
    with open(phoneme + '.json', 'w') as f:
        json.dump(json_pattern, f)
    

In [ ]:
def overlapping_hann_pattern(coord_list, pho_freq, disc_hann, total_time, step):
    """
    coord_list: list of coordinates, e.g. [11, 12, 13, 14]
    pho_freq: frequency of phoneme, e.g. 300
    disc_hann: discretization rate of Hanning window, default 12
    total_time: duration of pattern in ms, e.g. 392
    step: timestep for a new Hanning window to be added
    
    returns: list of data needed to create the .json in the right format
    """
    max_amp = 250                   # maximum amplitude, for our motors 250
    window = np.hanning(disc_hann)  # creating normalized hanning window
    cos_window = max_amp*window     # scaling to maximum amplitude
    
    nCoords = len(coord_list)
    bool_list = [False for i in range(nCoords)]   # list to keep track whether coordinate is 'done'
    print(total_time / disc_hann)                 # to check it's an integer
    print(step / disc_hann)                       # to check it's an integer
    disc_time = total_time // disc_hann           
    disc_step = step // disc_hann
    amp_list = [[0]*disc_time for _ in range(nCoords)]     # amplitude for each discretized timestep, for each coordinate
    # loop over all coordinates, and then over all timesteps, and assign amplitude
    for coord in range(nCoords): 
        for timestep in range(disc_time - sum(bool_list)*disc_step):
            amp_list[coord][timestep + sum(bool_list)*disc_step] = cos_window[timestep if timestep < disc_hann else 0]
        bool_list[coord] = True
    
    #NOTE: here it is assumed that for coordinate x, x+1 is simultaneously vibrating
    data = []
    for t in range(disc_time):
        iteration = {"iteration": [], "time": 5}  # 5 is default, can be changed.  
        for i in range(nCoords):
            if int(amp_list[i][t]) != 0:  # filter out amplitude = 0
                motor = {"coord": coord_list[i], "amplitude": int(amp_list[i][t]), "frequency": pho_freq}
                motor2 = {"coord": coord_list[i]+1, "amplitude": int(amp_list[i][t]), "frequency": pho_freq}
                iteration["iteration"].append(motor)
                iteration["iteration"].append(motor2)
        data.append(iteration)
                
    return data        

In [ ]:
def create_overlapping_hann_pattern(phoneme, coord_list, pho_freq, disc_hann, total_time, step):
    """
    phoneme: name of phoneme, e.g. 'OE'
    coord_list: list of coordinates, e.g. [11, 12, 13, 14]
    pho_freq: frequency of phoneme, e.g. 300
    disc_hann: discretization rate of Hanning window, default 24
    total_time: duration of pattern in ms, e.g. 392
    step: timestep for a new Hanning window to be added
    
    returns: creates .json file of phoneme
    """
    all_waves = overlapping_hann_window(coord_list, pho_freq, disc_hann, total_time, step)
    json_pattern = {"pattern" : all_waves}
    with open(phoneme + '.json', 'w') as f:
        json.dump(json_pattern, f)
    

Below, there is a helpful function to plot sinuses, afterwards the sinusiodal pattern generation functions are given. 

$y = A \sin(B(x - \phi)) + D$

In [ ]:
def plot_sinus(total_time, modulation, fraction, phi, dis):
    """
    total_time: total time of sinus in ms, e.g. 392
    modulation: modulation of wave in Hz, e.g. 30 
    fraction: fraction of the max amplitude of the motors to be the minimum of the wave, default 0.5
    phi: phase change, e.g. 0.4*(1 / modulation)
    dis: discretization rate, default for 092 time waves is 6, default for 392 time waves is 12
    
    returns: list of data needed to create the .json in the right format
    """

    #from input
    time = time / 1000 #s
    
    #known
    max_amp = 250 

    #can be calculated from input
    period = 1 / modulation
    B = 2*np.pi*modulation
    A = (max_amp - fraction*max_amp)/2 
    D = max_amp - A

    #for creating wave
    start = 0
    stop = time
    x = np.linspace(start, stop, int(time*1000/dis))

    #create amplitude list
    amplitude_list = []
    for i in range(len(x)):
        amplitude_list.append(A*np.sin(B*(x[i] - phi)) + D)

    #plotting
    plt.plot(x, A*np.sin(B*(x - phi)) + D)
    plt.title("wave")
    plt.xlabel("time(s)")
    plt.ylabel("amplitude")
    plt.show()


In [ ]:
def sin_modulation(total_time, modulation, fraction, phi, dis, coord_list, pho_freq, dynamic):
    """ 
    total_time: total time of sinus in ms, e.g. 392
    modulation: modulation of wave in Hz, e.g. 30 
    fraction: fraction of the max amplitude of the motors to be the minimum of the wave, default 0.5
    phi: phase change, e.g. 0.4*(1 / modulation)
    dis: discretization rate, default for 092 time waves is 6, default for 392 time waves is 12
    coord_list: list of coordinates, e.g. [12, 13, 14, 15]
    pho_freq: frequency of phoneme, e.g. 300
    dynamic: generate siple or dynamic pattern
    
    returns: makes a plot of how the sinus looks like
    """
    
    #from input
    if dynamic is True:
        time = total_time / 1000
    else:
        time = total_time
    
    #known
    max_amp = 250
    
    #can be calculated from input
    period = 1 / modulation
    B = 2*np.pi*modulation
    A = (max_amp - fraction*max_amp)/2 
    D = max_amp - A
    
    #for creating wave
    start = 0
    stop = time
    x = np.linspace(start, stop, int(time*1000/dis))
#     print(x)

    #create amplitude list
    amplitude_list = []
    for i in range(len(x)):
        amplitude_list.append((int) (A*np.sin(B*(x[i] - phi)) + D))

    data = []
    for i in range(len(amplitude_list)):
        iteration = {"iteration": [], "time": 10}
        motor = {"coord": coord_list[0], "amplitude": amplitude_list[i], "frequency": pho_freq}
        if len(coord_list) == 2:
            motor2 = {"coord": coord_list[1], "amplitude": amplitude_list[i], "frequency": pho_freq}
        if len(coord_list) == 3:
            motor2 = {"coord": coord_list[1], "amplitude": amplitude_list[i], "frequency": pho_freq}
            motor3 = {"coord": coord_list[2], "amplitude": amplitude_list[i], "frequency": pho_freq}
        if len(coord_list) == 4:
            motor2 = {"coord": coord_list[1], "amplitude": amplitude_list[i], "frequency": pho_freq}
            motor3 = {"coord": coord_list[2], "amplitude": amplitude_list[i], "frequency": pho_freq}
            motor4 = {"coord": coord_list[3], "amplitude": amplitude_list[i], "frequency": pho_freq}
        if len(coord_list) == 8:
            motor2 = {"coord": coord_list[1], "amplitude": amplitude_list[i], "frequency": pho_freq}
            motor3 = {"coord": coord_list[2], "amplitude": amplitude_list[i], "frequency": pho_freq}
            motor4 = {"coord": coord_list[3], "amplitude": amplitude_list[i], "frequency": pho_freq}
            motor5 = {"coord": coord_list[4], "amplitude": amplitude_list[i], "frequency": pho_freq}
            motor6 = {"coord": coord_list[5], "amplitude": amplitude_list[i], "frequency": pho_freq}
            motor7 = {"coord": coord_list[6], "amplitude": amplitude_list[i], "frequency": pho_freq}
            motor8 = {"coord": coord_list[7], "amplitude": amplitude_list[i], "frequency": pho_freq}
        iteration["iteration"].append(motor)
        if len(coord_list) == 2:
            iteration["iteration"].append(motor2)
        if len(coord_list) == 3:
            iteration["iteration"].append(motor2)
            iteration["iteration"].append(motor3)
        if len(coord_list) == 4:
            iteration["iteration"].append(motor2)
            iteration["iteration"].append(motor3)
            iteration["iteration"].append(motor4)
        if len(coord_list) == 8:
            iteration["iteration"].append(motor2)
            iteration["iteration"].append(motor3)
            iteration["iteration"].append(motor4)
            iteration["iteration"].append(motor5)
            iteration["iteration"].append(motor6)
            iteration["iteration"].append(motor7)
            iteration["iteration"].append(motor8)
        data.append(iteration)
        
    return data

In [ ]:
def create_simple_sin_modulation(phoneme, total_time, modulation, fraction, phi, dis, coord_list, pho_freq):
    """
    phoneme: name of the phoneme, e.g. 'B'
    total_time: total time of sinus in ms, e.g. 392
    modulation: modulation of wave in Hz, e.g. 30 
    fraction: fraction of the max amplitude of the motors to be the minimum of the wave, default 0.5
    phi: phase change, e.g. 0.4*(1 / modulation)
    dis: discretization rate, default for 092 time waves is 6, default for 392 time waves is 12
    coord_list: list of coordinates, e.g. [12, 13, 14, 15]
    pho_freq: frequency of phoneme, e.g. 300
    
    returns: creates a .json file of phoneme
    """
    all_waves = sin_modulation(total_time, modulation, fraction, phi, dis, coord_list, pho_freq, False)
    json_pattern = {"pattern" : all_waves}
    with open(phoneme + '.json','w') as f:
        json.dump(json_pattern, f)

## Example of usage for UU pattern

In [ ]:
# UU
modulation = 60
all_waves = sin_modulation(60, 30, 0.5, 0, 6, [31,41], 300, True)
all_waves += (sin_modulation(60, 30, 0.5, 0.2*(1 / modulation), 6, [31,41], 300, True))
all_waves += (sin_modulation(60, 30, 0.5, 0.4*(1 / modulation), 6, [33,43], 300, True))
all_waves += (sin_modulation(60, 30, 0.5, 0.6*(1 / modulation), 6, [34,44], 300, True))
all_waves += (sin_modulation(80, 30, 0.5, 0.4*(1 / modulation), 6, [32,42], 300, True))
all_waves += (sin_modulation(80, 30, 0.5, 0, 6, [31,41], 300, True))
all_waves += (sin_modulation(60, 30, 0.5, 0.2*(1 / modulation), 6, [24], 300, True))
all_waves += (sin_modulation(60, 30, 0.5, 0.4*(1 / modulation), 6, [23], 300, True))
json_pattern = {"pattern": all_waves}
with open('UU.json','w') as f:
    json.dump(json_pattern, f)

In [ ]:
# most implementations of patterns used listed:
modulation = 60

#B
all_waves = create_simple_sin_modulation('B', 0.092, 30, 0.5, 0, 6, [15,16,25,26], 300)

#M
all_waves = create_simple_sin_modulation('M',0.392, 8, 0.5, 0, 12, [15,16,25,26], 60)

#J
all_waves = create_simple_sin_modulation('J', 0.392, 8, 0.5, 0, 12, [11,16,21,26], 300)

#D
all_waves = create_simple_sin_modulation('D', 0.092, 30, 0.5, 0, 6, [33,34,43,44], 300)

#G
all_waves = create_simple_sin_modulation('G', 0.092, 30, 0.5, 0, 6, [11,12,21,22], 300)

#V
all_waves = create_simple_sin_modulation('V', 0.392, 8, 0.5, 0, 12, [16,26,36,46], 300)

#DH
all_waves = create_simple_sin_modulation('DH', 0.392, 8, 0.5, 0, 12, [13,14,23,24], 300)

#Z
all_waves = create_simple_sin_modulation('Z', 0.392, 8, 0.5, 0, 12, [11,21,31,41], 300)

#ZH
all_waves = create_simple_sin_modulation('ZH', 0.392, 8, 0.5, 0, 12, [31,32,41,42], 60)

#N
all_waves = create_simple_sin_modulation('N', 0.392, 8, 0.5, 0, 12, [33,34,43,44], 60)

#NG
all_waves = create_simple_sin_modulation('NG', 0.392, 8, 0.5, 0, 12, [11,12,21,22], 60)

#W
all_waves = create_simple_sin_modulation('W', 0.392, 8, 0.5, 0, 12, [13,14,15,16,33,34,35,36], 60)

#L
all_waves = create_simple_sin_modulation('L', 0.392, 30, 0.5, 0, 12, [35,36,45,46], 300)

#R
all_waves = create_simple_sin_modulation('R', 0.392, 30, 0.5, 0, 12, [31,32,41,42], 300)